In [ ]:
from see.Segment_Fitness import FitnessFunction
from see.Segmentors import segmentor
from see.ColorSpace import colorspace
from see.Workflow import workflow
from see.Segment_Fitness import segment_fitness
from see import base_classes, GeneticSearch
import copy
import numpy as np
FitnessFunction(datas[3].mask,datas[1].mask)

In [2]:
def GeneratePairs(size): #Generates each pair in a set
    pairs=[]
    for i in range(size):
        for j in range(i+1,size):
            pairs.append([i,j])
    return pairs
GeneratePairs(4)

[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]]

In [ ]:

def UncertaintyValue(segmenters,data): #Compute uncertainty value
    #Generate N copies of image
    dataCopies=[copy.deepcopy(data) for i in range(len(segmenters))]

    #Create segmenters
    segs=[workflow(i) for i in segmenters]

    #Run each segmenter on a copy of an image
    dataCopies=[segs[i].runAlgo(dataCopies[i]) for i in range(len(segs))]

    #Return the mean difference between each pairwise prediction
    uncertainty=mean([FitnessFunction(dataCopies[i[0]],dataCopies[i[1]]) for i in GeneratePairs(len(dataCopies))])
    return uncertainty

UncertaintyValue

In [ ]:
def ALSearch(segmenters,dataSet): #Search data set for image with highest uncertainty value
    #Generate uncertainty value for each image
    uncertainties=[UncertaintyValue(segmenters,data) for data in dataSet]

    #Return copy of image with most uncertainty (should this be argmax?)
    return np.argmin(uncertainties) #copy.deepcopy(dataSet[np.argmin(uncertainties)])

In [ ]:
def ActiveLearning(trainingDataSet,testData,iterations=10):
    #Initialize selected training set and fitness tracking
    selectedTrainSet=[]
    testFitness=[]

    #Copy training set to unselected set
    unselectedTrainingData=copy.deepcopy(trainingDataSet)

    #Select a single image from training set to initialize active training set and remove from unselected set
    selectedInt=np.random.randint(0,len(unselectedTrainingData))
    selectedTrainSet.append(unselectedTrainingData[selectedInt])
    del unselectedTrainingData[selectedInt]

    #Define workflow
    workflow.addalgos([colorspace, segmentor, segment_fitness])

    #Begin loop over active learning iterations
    for i in range(iterations): 
        #Execute first round of evolution
        my_evolver = GeneticSearch.Evolver(workflow, selectedTrainSet, pop_size=100)
        population = my_evolver.run(ngen=10)

        #Generate model ensemble from quality models in the population (HoF)
        ensemble=[]
        ensemble=my.evolver.hof[:min(10,len(my_evolver.hof))]

        #Use ALSearch to maximize uncertainty over unselected training set
        selectedImage=ALSearch(ensemble,unselectedTrainingData)
        

        #Add new sample to training set and remove from unselected
        selectedTrainSet.append(unselectedTrainingData[selectedImage])
        del unselectedTrainingData[selectedImage]

        #Record current best performance on test data and append to tracking list
        bestFitness=min([])
        testFitness.append(bestFitness)
        

    #Return best model found, best fitness found, and fitness list
    pass